#Gjuha e shenjave shqip për instalim lokal me YOLOv8

## Importimi dhe instalimi i mvarësive 

In [ ]:
# %%
# Install required packages (only need to run once)
!pip install ultralytics opencv-python matplotlib numpy scikit-learn tqdm pyyaml --quiet

In [1]:
# Import libraries
from ultralytics import YOLO
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image
import yaml
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm

## Ngarkimi dhe pregatitja e të dhënave

In [2]:
# %%
# Configuration
DATASET_DIR = r"C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images"  # Root directory of your dataset
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1
SEED = 42  # For reproducible splits

# Create YOLOv8 dataset structure
def create_yolov8_dataset_structure(dataset_dir, train_ratio, val_ratio, test_ratio, seed=42):
    """
    Organizes the dataset into YOLOv8 expected structure:
    - dataset/
        - train/
            - images/
            - labels/
        - val/
            - images/
            - labels/
        - test/
            - images/
            - labels/
    """
    
    # Create directories if they don't exist
    os.makedirs(os.path.join(dataset_dir, "train", "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "train", "labels"), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "val", "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "val", "labels"), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "test", "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "test", "labels"), exist_ok=True)
    
    # Get list of all image files (assuming .jpg format)
    image_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]
    
    # Split into train, val, test
    train_files, temp_files = train_test_split(image_files, train_size=train_ratio, random_state=seed)
    val_files, test_files = train_test_split(temp_files, test_size=test_ratio/(val_ratio+test_ratio), random_state=seed)
    
    # Helper function to copy files
    def copy_files(files, split):
        for file in tqdm(files, desc=f"Copying {split} files"):
            # Copy image
            src_img = os.path.join(dataset_dir, file)
            dst_img = os.path.join(dataset_dir, split, "images", file)
            shutil.copy(src_img, dst_img)
            
            # Copy corresponding label file
            label_file = os.path.splitext(file)[0] + '.txt'
            src_label = os.path.join(dataset_dir, label_file)
            dst_label = os.path.join(dataset_dir, split, "labels", label_file)
            if os.path.exists(src_label):
                shutil.copy(src_label, dst_label)
    
    # Copy files to respective directories
    copy_files(train_files, "train")
    copy_files(val_files, "val")
    copy_files(test_files, "test")
    
    print("Dataset structure created successfully!")

# Create the dataset structure
create_yolov8_dataset_structure(DATASET_DIR, TRAIN_RATIO, VAL_RATIO, TEST_RATIO, SEED)

# Create dataset YAML file
data_yaml = {
    'train': os.path.join(DATASET_DIR, 'train', 'images'),
    'val': os.path.join(DATASET_DIR, 'val', 'images'),
    'test': os.path.join(DATASET_DIR, 'test', 'images'),
    'nc': 6,  # Number of classes (Albanian alphabet has 36 letters)
    'names': [
        'C', 'E', 'G', 'L', 'O', 'V'
    ]
}

# Save the YAML file
yaml_path = os.path.join(DATASET_DIR, 'albanian_signs.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"Dataset YAML file created at {yaml_path}")

Copying test files: 100%|████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 83.76it/s]

Dataset structure created successfully!
Dataset YAML file created at C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\albanian_signs.yaml


Ngarkimi dhe përshtatja e modelit Yolo V8

In [3]:
# %%
# Load a pretrained YOLOv8n model (you can choose different sizes: n, s, m, l, x)
model = YOLO('yolov8n.pt')  # Load a pretrained model (recommended for training)

# Display model architecture (optional)
model.info()

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

## Trajnimi i modelit

In [4]:
import torch  # Add this import
from ultralytics import YOLO
import os

# %%
# Training configuration
EPOCHS = 50
BATCH_SIZE = 8
LEARNING_RATE = 0.01
IMG_SIZE = (1024)
PATIENCE = 10  # Early stopping patience
DEVICE = '0' if torch.cuda.is_available() else 'cpu'  # Use GPU if available

# Train the model
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    lr0=LEARNING_RATE,
    imgsz=IMG_SIZE,
    patience=PATIENCE,
    device=DEVICE,
    project='albanian_signs_detection',
    name='train1',
    save=True,
    save_period=5,  # Save checkpoint every 5 epochs
    visualize=True,  # Visualize training progress
    augment=True,  # Apply mosaic augmentation
    cache=False  # Cache images for faster training (requires more RAM)
)

print("Training completed!")

Ultralytics 8.3.115  Python-3.10.16 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\albanian_signs.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=(1024, 768), save=True, save_period=5, cache=False, device=cpu, workers=8, project=albanian_signs_detection, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fal

train: Scanning C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\train\labels... 10 images, 0 backgrou

train: New cache created: C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\train\labels.cache
val: Fast image access  (ping: 0.30.0 ms, read: 446.788.9 MB/s, size: 619.5 KB)



C:\Users\hara11\AppData\Local\anaconda3\envs\gjsh_lili\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\val\labels... 3 images, 0 backgrounds, 

val: New cache created: C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\val\labels.cache


...

      27/50         0G     0.9558      2.709      1.774          6       1024: 100%|██████████| 2/2 [00:13<00:00,  6.84
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<0

                   all          3          3     0.0134          1      0.663      0.406
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 17, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



27 epochs completed in 0.127 hours.
Optimizer stripped from albanian_signs_detection\train14\weights\last.pt, 6.3MB
Optimizer stripped from albanian_signs_detection\train14\weights\best.pt, 6.3MB

Validating albanian_signs_detection\train14\weights\best.pt...
Ultralytics 8.3.115  Python-3.10.16 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0


                   all          3          3     0.0165          1       0.58      0.522
                     C          1          1     0.0159          1      0.249      0.224
                     E          1          1     0.0143          1      0.497      0.448
                     G          1          1     0.0192          1      0.995      0.895
Speed: 8.6ms preprocess, 622.6ms inference, 0.0ms loss, 159.5ms postprocess per image
Results saved to albanian_signs_detection\train14
Training completed!


## Vlerësimi i performancës 

In [5]:
# Load the best model from training
best_model_path = os.path.join('albanian_signs_detection', 'train1', 'weights', 'best.pt')
best_model = YOLO(best_model_path)

# Evaluate model performance on the test set
metrics = best_model.val(
    data=yaml_path,
    split='test',
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    conf=0.25,  # Confidence threshold
    iou=0.45,  # IoU threshold
    device=DEVICE
)

# Print evaluation metrics - UPDATED FOR YOLOv8.3+
print("\nEvaluation Metrics:")
print(f"mAP@0.5: {metrics.box.map50:.4f}")  # mAP@0.5
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")  # mAP@0.5:0.95
print(f"Mean Precision: {metrics.box.mp:.4f}")
print(f"Mean Recall: {metrics.box.mr:.4f}")

# Plot some predictions on test images
test_images_dir = os.path.join(DATASET_DIR, 'test', 'images')
test_images = [os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir) if f.endswith('.jpg')][:5]  # First 5 images

for img_path in test_images:
    results = best_model(img_path)
    for r in results:
        im_array = r.plot()  # Plot a BGR numpy array of predictions
        im = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        # Display in notebook
        plt.figure(figsize=(10, 10))
        plt.imshow(im)
        plt.axis('off')
        plt.show()

WARNING updating to 'imgsz=1024'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'
Ultralytics 8.3.115  Python-3.10.16 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
Model summary (fused): 72 layers, 3,012,668 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.30.0 ms, read: 292.62.6 MB/s, size: 502.6 KB)


val: Scanning C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\test\labels... 2 images, 0 backgrounds,

val: New cache created: C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\test\labels.cache



C:\Users\hara11\AppData\Local\anaconda3\envs\gjsh_lili\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all          2          2          0          0          0          0
                     B          1          1          0          0          0          0
                     C          1          1          0          0          0          0
Speed: 9.9ms preprocess, 285.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val4

Evaluation Metrics:
mAP@0.5: 0.0000
mAP@0.5:0.95: 0.0000
Mean Precision: 0.0000
Mean Recall: 0.0000

image 1/1 C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\test\images\cvimg_14.jpg: 640x480 (no detections), 187.2ms
Speed: 8.7ms preprocess, 187.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)


<Figure size 1000x1000 with 1 Axes>


image 1/1 C:\AI\SingLanguageAI\Albanian-Sign-Language-Detection\data\images\test\images\cvimg_6.jpg: 640x480 (no detections), 125.8ms
Speed: 9.0ms preprocess, 125.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


<Figure size 1000x1000 with 1 Axes>

## Testimi me kamerë

In [6]:
def run_webcam_inference(model, conf_threshold=0.5):
    """
    Run real-time inference using webcam
    Args:
        model: YOLOv8 model
        conf_threshold: Confidence threshold for detection
    """
    cap = cv2.VideoCapture(0)  # 0 for default webcam
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    print("Starting webcam inference... Press 'q' to quit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break
        
        # Run inference
        results = model(frame, conf=conf_threshold)
        
        # Visualize results
        annotated_frame = results[0].plot()
        
        # Display the annotated frame
        cv2.imshow("Albanian Sign Detection", annotated_frame)
        
        # Break the loop on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()

# Run webcam inference with the best model
run_webcam_inference(best_model, conf_threshold=0.5)

Starting webcam inference... Press 'q' to quit.

0: 480x640 (no detections), 196.2ms
Speed: 5.3ms preprocess, 196.2ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 135.9ms
Speed: 5.3ms preprocess, 135.9ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 127.4ms
Speed: 4.3ms preprocess, 127.4ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 126.9ms
Speed: 3.9ms preprocess, 126.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 127.2ms
Speed: 3.9ms preprocess, 127.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 125.4ms
Speed: 4.4ms preprocess, 125.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 131.2ms
Speed: 3.8ms preprocess, 131.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480